# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `01_materials/labs/2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [1]:
# Load the environment variables using dotenv.
%load_ext dotenv
%dotenv 

In [2]:
import dask.dataframe as dd

c:\Users\sijia\miniconda3\envs\dsi_participant\lib\site-packages\dask\dataframe\_pyarrow_compat.py:17: FutureWarning: Minimal version of pyarrow will soon be increased to 14.0.1. You are using 11.0.0. Please consider upgrading.
  warnings.warn(
C:\Users\sijia\AppData\Local\Temp\ipykernel_2956\676544213.py:1: DeprecationWarning: The current Dask DataFrame implementation is deprecated. 
In a future release, Dask DataFrame will use new implementation that
contains several improvements including a logical query planning.
The user-facing DataFrame API will remain unchanged.

The new implementation is already available and can be enabled by
installing the dask-expr library:

    $ pip install dask-expr

and turning the query planning option on:

    >>> import dask
    >>> dask.config.set({'dataframe.query-planning': True})
    >>> import dask.dataframe as dd

API documentation for the new implementation is available at
https://docs.dask.org/en/stable/dask-expr-api.html

Any feedback can be 

+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [3]:
import os
from glob import glob

# Load the PRICE_DATA environment variable
price_data_dir = os.getenv('PRICE_DATA')

# Use glob to find all parquet files in PRICE_DATA directory, excluding directories
parquet_files = [f for f in glob(os.path.join(price_data_dir, '**/*.parquet'), recursive=True) if os.path.isfile(f)]

# Check that parquet_files only includes file paths, not directories
print(parquet_files)

# Load the data from parquet files into a Dask DataFrame
dd_px = dd.read_parquet(parquet_files).set_index("ticker")

['../../05_src/data/prices\\A\\A_2000.parquet\\part.0.parquet', '../../05_src/data/prices\\A\\A_2001.parquet\\part.0.parquet', '../../05_src/data/prices\\A\\A_2002.parquet\\part.0.parquet', '../../05_src/data/prices\\A\\A_2003.parquet\\part.0.parquet', '../../05_src/data/prices\\A\\A_2004.parquet\\part.0.parquet', '../../05_src/data/prices\\A\\A_2005.parquet\\part.0.parquet', '../../05_src/data/prices\\A\\A_2006.parquet\\part.0.parquet', '../../05_src/data/prices\\A\\A_2007.parquet\\part.0.parquet', '../../05_src/data/prices\\A\\A_2008.parquet\\part.0.parquet', '../../05_src/data/prices\\A\\A_2009.parquet\\part.0.parquet', '../../05_src/data/prices\\A\\A_2010.parquet\\part.0.parquet', '../../05_src/data/prices\\A\\A_2011.parquet\\part.0.parquet', '../../05_src/data/prices\\A\\A_2012.parquet\\part.0.parquet', '../../05_src/data/prices\\A\\A_2013.parquet\\part.0.parquet', '../../05_src/data/prices\\A\\A_2014.parquet\\part.0.parquet', '../../05_src/data/prices\\A\\A_2015.parquet\\part.0.p

c:\Users\sijia\miniconda3\envs\dsi_participant\lib\site-packages\dask\dataframe\core.py:5517: UserWarning: New index has same name as existing, this is a no-op.
  warnings.warn(


For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Adjusted Close:
    
    - `returns`: (Adj Close / Adj Close_lag) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [4]:
import numpy as np

# Define the metadata to include all columns in dd_px, including 'hi_lo_range' and 'returns'
meta = {
    'Date': 'datetime64[ns]',
    'Open': 'f8',
    'High': 'f8',
    'Low': 'f8',
    'Close': 'f8',
    'Adj Close': 'f8',
    'Volume': 'f8',
    'sector': 'object',
    'subsector': 'object',
    'year': 'int64',
    'Close_lag_1': 'f8',
    'Adj_Close_lag_1': 'f8',
    'returns': 'f8',
    'hi_lo_range': 'f8',
}

# Perform all transformations, including adding lagged columns, returns, and hi_lo_range, within a single groupby apply
dd_px = dd_px.groupby("ticker", group_keys=False).apply(
    lambda x: x.assign(
        Close_lag_1=x['Close'].shift(1),
        Adj_Close_lag_1=x['Adj Close'].shift(1),
        returns=(x['Adj Close'] / x['Adj Close'].shift(1)) - 1,
        hi_lo_range=x['High'] - x['Low']
    ),
    meta=meta
)

# Assign the resulting DataFrame to `dd_feat`
dd_feat = dd_px

+ Convert the Dask data frame to a pandas data frame. 
+ Add a rolling average return calculation with a window of 10 days.
+ *Tip*: Consider using `.rolling(10).mean()`.

(3 pt)

In [6]:
# Convert the Dask DataFrame to a Pandas DataFrame for additional rolling calculations
pd_feat = dd_feat.compute()

# Reset the index to avoid issues with duplicate labels
pd_feat = pd_feat.reset_index()

# Add a rolling average return calculation with a 10-day window, grouped by 'ticker'
# This approach ensures rolling calculation operates independently for each group
pd_feat['rolling_avg_return_10d'] = (
    pd_feat.groupby('ticker')['returns']
    .rolling(10, min_periods=1)
    .mean()
    .reset_index(level=0, drop=True)  # Drop the duplicate index level created by groupby
)

# Display the processed data
pd_feat.head()

,index,ticker,Date,Open,High,Low,Close,Adj Close,Volume,sector,subsector,year,Close_lag_1,Adj_Close_lag_1,returns,hi_lo_range,rolling_avg_return_10d
0,0,HUM,2017-01-03,202.869995,204.000000,195.000000,197.610001,186.429749,2784600,Health Care,Managed Health Care,2017,NaN,NaN,NaN,9.000000,NaN
1,1,HUM,2017-01-04,198.410004,199.440002,195.910004,197.779999,186.590149,1512400,Health Care,Managed Health Care,2017,197.610001,186.429749,0.000860,3.529999,0.000860
2,2,HUM,2017-01-05,198.449997,201.589996,197.190002,200.729996,189.373260,1826200,Health Care,Managed Health Care,2017,197.779999,186.590149,0.014916,4.399994,0.007888
3,3,HUM,2017-01-06,200.500000,202.889999,199.619995,201.729996,190.316696,1139700,Health Care,Managed Health Care,2017,200.729996,189.373260,0.004982,3.270004,0.006919
4,4,HUM,2017-01-09,201.050003,203.470001,199.750000,202.990005,191.505432,1063800,Health Care,Managed Health Care,2017,201.729996,190.316696,0.006246,3.720001,0.006751


Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?
+ Would it have been better to do it in Dask? Why?

(1 pt)

**1. Was it necessary to convert to Pandas to calculate the moving average return?**

> No, converting to Pandas was not strictly necessary to calculate the moving average return. Dask can handle rolling calculations like moving averages directly on Dask DataFrames, including group-based rolling operations, which would avoid needing to load the entire dataset into memory as with Pandas.


**2. Would it have been better to do it in Dask? Why?**

> Yes, it would generally be better to perform the moving average calculation in Dask, especially if working with a large dataset that doesn’t fit comfortably into memory. Here’s why:
> - Memory Efficiency: Dask processes data in smaller, manageable chunks, which helps prevent memory overload when working with large datasets. This contrasts with Pandas, which requires the entire DataFrame in memory.
> - Parallel Processing: Dask can distribute calculations across multiple cores and partitions, making it faster and more efficient for large-scale data processing tasks.
> - Reduced Conversions: Converting between Dask and Pandas repeatedly can add unnecessary overhead and complexity to code. By keeping everything in Dask, you retain the efficiency of lazy evaluation and avoid time-consuming conversions.

## Criteria

The [rubric](./assignment_1_rubric_clean.xlsx) contains the criteria for grading.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.